In [6]:
from abc import ABC, abstractmethod
from re import L
from typing import List
from sklearn.metrics import accuracy_score
import numpy as np
import tqdm
from typing import List
from torchvision.datasets import MNIST

In [7]:
def to_numpy(x):
  x = np.array(x).flatten()
  return x / 255

In [8]:
train_dataset = MNIST(".", download=True, train=True, transform=to_numpy)
valid_dataset = MNIST(".", download=True, train=False, transform=to_numpy)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [ ]:
len(train_dataset), len(valid_dataset)

In [10]:
def weights_initialization(input_size, output_size):
    # USE BETTER INITIALIZATION
    return np.zeros((input_size, output_size))

In [11]:
class Layer(ABC):

    @abstractmethod
    def forward(self, x):
        raise NotImplementedError

    @abstractmethod
    def backward(self, out, learning_rate):
        raise NotImplementedError

In [12]:
class LinearLayer(Layer):
    
    def __init__(self, weights, biases):
        self.weights = weights
        self.biases = biases
        
    def forward(self, x):
        self.x = x
        return x.dot(self.weights) + self.biases

    def backward(self, output_grad, learning_rate):
      x_grad = self.x.T.dot(output_grad)
      weights = self.weights.copy()
      self.weights -= learning_rate * x_grad
      self.bias -= learning_rate * output_grad
      return np.dot(output_grad, np.transpose(self.weights))    

In [15]:
class ReLU:
  def __init__(self):
    self.data = None

  def forward(self, x):
    self.data = x
    x[x <= 0] = 0
    return x


  def backward(self, output_grad, learning_rate) : 
    mask = (self.data >0).astype(np.float32)
    return mask * output_grad


In [16]:
class Softmax:
  def __init__(self):
    self.data = None

  def forward(self, x):
    expx = np.exp(x - np.max(x))
    self.data = expx / expx.sum()
    return self.data

  def backward(self, output_grad, learning_rate):
    self.data[output_grad] -= 1
    return self.data

In [19]:
class Graph:

    def __init__(self, layers: List[Layer], learning_rate: float) -> None:
        self.layers = layers
        self.lr = learning_rate

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, y):
        for layer in reversed(self.layers):
            y = layer.backward(y, self.lr)
        return y

In [20]:
MNIST(".", download=True)

Dataset MNIST
    Number of datapoints: 60000
    Root location: .
    Split: Train